In [16]:
import attr
import numpy as np
import pandas as pd

import abct_comp_ner_utils.brackets as br

with open("../../comp-proto/NER-test-result_2022-08-22/NER-test-result_2022-08-22.txt") as file:
    data = tuple(
        attr.asdict(item)
        for item in br.parse_test_analysis_file(file)
    )

In [17]:
df = pd.DataFrame(data)
# filter out irrelevant examples
df = df[df["cont_role"] != "none"]

# set IDs as indices
df.index = pd.Index(df["data_ref"].map(lambda i: i["ID"]), name = "ID")

# extract the info of whether predictions are correct or not
df["is_correct"] = df["annot_labels"].map(lambda i: "correct" in i)

In [18]:
from typing import Iterable, Any

def get_prej_cont_rel(comps: Iterable[dict[str, Any]]) -> str:
    list_cont = tuple(
        c for c in comps 
        if c["label"] == "cont"
    )
    list_prej = tuple(
        c for c in comps 
        if c["label"] == "prej"
    )
    if list_cont:
        # get the start number
        cont_start = min(100000, 100000, *(item["start"] for item in list_cont))
        prej_end: int = max(0, 0, *(item["end"] for item in list_prej))

        if prej_end <= cont_start:
            return "prej < cont"
        else:
            return "cont <= prej"
    else:
        return "cont_not_appearing"

df["prej-cont-rel"] = df["data_ref"].map(
    lambda i: get_prej_cont_rel(i["comp"])
)

In [31]:
# 全体の正答率
# contがあっているかどうか，というよりは，diff, prej全体を含めてあっているかどうかの率。
# なので，やや低めに出る。
df["is_correct"].mean()

0.5595667870036101

In [19]:
# prej-contの順番・有無による予測の成否を調べる。
# まずは分布
df["is_correct"].groupby(df["prej-cont-rel"]).count()

prej-cont-rel
cont <= prej          122
cont_not_appearing     75
prej < cont            80
Name: is_correct, dtype: int64

In [20]:
# 成否の割合
df["is_correct"].groupby(df["prej-cont-rel"]).mean()

# contrastがcovertならばprecisionがやや高い。あとはあんまり相関がない。

prej-cont-rel
cont <= prej          0.508197
cont_not_appearing    0.666667
prej < cont           0.537500
Name: is_correct, dtype: float64

In [33]:
# 次に，contrastの文法的役割による予測の成否を調べる。
# まずは分布
df["is_correct"].groupby(df["cont_role"]).count()

# Sが多い。次にcovertなもの。
# O, ADVは少ない。

cont_role
ADV        14
O          15
PRED        2
S         127
S1         16
covert     73
rchead     30
Name: is_correct, dtype: int64

In [23]:
# 次に割合

df["is_correct"].groupby(df["cont_role"]).mean()

# contrastが主語，目的語，二重主語の第一主語（S1），covertの場合にはprecisionが高い。
# あとは独立。
# rchead（関係節）については，アノテーションの関係で0%。

cont_role
ADV       0.500000
O         0.600000
PRED      0.500000
S         0.590551
S1        0.750000
covert    0.698630
rchead    0.000000
Name: is_correct, dtype: float64

In [30]:
table = df.pivot_table(
    values = "is_correct",
    index = ["cont_role"],
    columns = ["prej-cont-rel"],
    aggfunc = {"is_correct": ["count", "mean"]},
)
table

# Oの絶対数がそもそも少ない

count                                        mean  \
prej-cont-rel cont <= prej cont_not_appearing prej < cont cont <= prej   
cont_role                                                                
ADV                    8.0                3.0         3.0     0.500000   
O                      2.0                1.0        12.0     0.500000   
PRED                   NaN                NaN         2.0          NaN   
S                     76.0                8.0        43.0     0.578947   
S1                    12.0                NaN         4.0     0.666667   
covert                 6.0               61.0         6.0     0.833333   
rchead                18.0                2.0        10.0     0.000000   

                                              
prej-cont-rel cont_not_appearing prej < cont  
cont_role                                     
ADV                     0.666667    0.333333  
O                       0.000000    0.666667  
PRED                         NaN    0.500000  
S                       0.250000    0.674419  
S1                           NaN    1.000000  
covert                  0.754098    0.000000  
rchead                  0.000000    0.000000